In [1]:
import pandas as pd

In [2]:
df_raw = pd.read_csv("./toxic.csv")
df_raw.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
TOXIC_LABELS = df_raw.columns.tolist()[2:]
toxic_items = df_raw[df_raw[TOXIC_LABELS].sum(axis=1) > 0]
ok_items = df_raw[df_raw[TOXIC_LABELS].sum(axis=1) == 0]

In [4]:
sample_size = 15000

In [5]:
df = pd.concat([
    toxic_items.sample(sample_size),
    ok_items.sample(sample_size)
])

In [6]:
df["label"] = (df[TOXIC_LABELS].sum(axis=1) > 0).astype(int)
df

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,label
144119,081a57601f9f54b0,er no he's 6 ft 4 ACTUALLY so get bent.if he ...,1,0,0,0,0,0,1
76827,cdbdd5f8d3d12442,lonely\n\nGo outside and make a friend. You'v...,0,0,0,0,1,0,1
66774,b2ade468ffdf9aff,"You are a troublemaker, a liar, and a maliciou...",1,0,0,0,0,0,1
49866,855269851f35946c,you are part of isis \n\nCouple of days ago u ...,1,0,0,1,0,0,1
75178,c924ea376dd6eb78,bomb is going to blow in your country go save ...,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
121862,8bf0349959d50fb7,"""\n\nSphinx Facing Leo vs Orion Rising - Witne...",0,0,0,0,0,0,0
64753,ad4b63c43d8da86f,icarly\n\nHey can you revert this edit \nhttp:...,0,0,0,0,0,0,0
44930,781c8ddcc4140ea2,"""\nThere's nothing wrong with being bold, unle...",0,0,0,0,0,0,0
133906,cc5983ea4556d740,"I dont even remember READING this, let alone e...",0,0,0,0,0,0,0


In [7]:
from datasets import Dataset

toxic_dataset = Dataset.from_pandas(df).train_test_split(test_size=0.2)
toxic_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'label', '__index_level_0__'],
        num_rows: 24000
    })
    test: Dataset({
        features: ['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate', 'label', '__index_level_0__'],
        num_rows: 6000
    })
})

In [8]:
from transformers import DistilBertTokenizer

bert_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize(data):
    return bert_tokenizer(data["comment_text"], truncation=True)

In [9]:
toxic_dataset_tokenized = toxic_dataset.map(tokenize)

Map:   0%|          | 0/24000 [00:00<?, ? examples/s]

Map:   0%|          | 0/6000 [00:00<?, ? examples/s]

In [10]:
pd.DataFrame(toxic_dataset_tokenized["test"]).head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,label,__index_level_0__,input_ids,attention_mask
0,3a537159742790f8,Speedy Deletion of Titans (Webcomic)\nFor cryi...,1,0,0,0,1,0,1,147318,"[101, 26203, 3972, 20624, 2239, 1997, 13785, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,3b3ed9cb4c17dd2a,WOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOOO!...,1,0,1,0,0,0,1,22439,"[101, 15854, 9541, 9541, 9541, 9541, 9541, 954...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,2858c07e8d49f2fc,"You blanked articles to warez groups, dumb ass...",1,0,1,0,1,0,1,103221,"[101, 2017, 8744, 2098, 4790, 2000, 16283, 248...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,30b0705e5c285c30,""" December 2005 (UTC)\nHellooo, Macheath, who'...",0,0,0,0,0,0,0,18446,"[101, 1000, 2285, 2384, 1006, 11396, 1007, 759...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,1abcdd3b4de39ae2,"""\n\nIt's a profound problem when administrato...",0,0,1,0,0,0,1,10112,"[101, 1000, 2009, 1005, 1055, 1037, 13769, 329...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [11]:
from transformers import DataCollatorWithPadding

collator = DataCollatorWithPadding(bert_tokenizer)

In [12]:
import evaluate

metric = evaluate.load("accuracy")
def calc_metrics(evaluation):
    logits, labels = evaluation
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [13]:
from transformers import TrainingArguments, Trainer
import numpy as np

In [14]:
batch_size = 64
epochs = 3

In [15]:
train_args = TrainingArguments(
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    output_dir="./output/models",
    logging_strategy="epoch",
    logging_dir="./output/log",
)

In [16]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

bert_classification_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
bert_classification_model.config.id2label = {0: "Ok", 1: "Toxic"}
bert_classification_model.config.label2id = {"Ok": 0, "Toxic": 1}

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
trainer = Trainer(
    model=bert_classification_model,
    processing_class=bert_tokenizer,
    args=train_args,
    train_dataset=toxic_dataset_tokenized["train"],
    eval_dataset=toxic_dataset_tokenized["test"],
    compute_metrics=calc_metrics,
    data_collator=collator,
)

In [18]:
trainer.evaluate()

{'eval_loss': 0.689630389213562,
 'eval_model_preparation_time': 0.001,
 'eval_accuracy': 0.5,
 'eval_runtime': 15.6863,
 'eval_samples_per_second': 382.5,
 'eval_steps_per_second': 5.992}

In [19]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Accuracy
1,0.189200,0.156079,0.001000,0.939000
2,0.094800,0.165607,0.001000,0.944000
3,0.041400,0.225799,0.001000,0.941667


TrainOutput(global_step=1125, training_loss=0.10846669430202908, metrics={'train_runtime': 590.7599, 'train_samples_per_second': 121.877, 'train_steps_per_second': 1.904, 'total_flos': 8949514011624192.0, 'train_loss': 0.10846669430202908, 'epoch': 3.0})

In [20]:
trainer.save_model()

In [21]:
api_key = 'hf_foobarbaz'
path = 'foo/bar'

In [22]:
trainer.model.push_to_hub(
    repo_id=path, token=api_key
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ZhengjunHUO/distilbert-toxicity-classifier/commit/489488aeff2d3447119e8fc98472246506a70213', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='489488aeff2d3447119e8fc98472246506a70213', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ZhengjunHUO/distilbert-toxicity-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='ZhengjunHUO/distilbert-toxicity-classifier'), pr_revision=None, pr_num=None)

In [23]:
trainer.processing_class.push_to_hub(
    repo_id=path, token=api_key
)

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/ZhengjunHUO/distilbert-toxicity-classifier/commit/489488aeff2d3447119e8fc98472246506a70213', commit_message='Upload tokenizer', commit_description='', oid='489488aeff2d3447119e8fc98472246506a70213', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ZhengjunHUO/distilbert-toxicity-classifier', endpoint='https://huggingface.co', repo_type='model', repo_id='ZhengjunHUO/distilbert-toxicity-classifier'), pr_revision=None, pr_num=None)

In [24]:
from transformers import pipeline

t_classifier = pipeline(
    'text-classification',
    model=trainer.model,
    tokenizer=bert_tokenizer,
    use_fast=True,
    top_k=None
)

Device set to use cuda:0


In [25]:
t_classifier('whoever wrote this is a waste of space')

[[{'label': 'Toxic', 'score': 0.9503776431083679},
  {'label': 'Ok', 'score': 0.04962233826518059}]]

In [26]:
t_classifier('Such an Idiot person')

[[{'label': 'Toxic', 'score': 0.9975517392158508},
  {'label': 'Ok', 'score': 0.002448315266519785}]]

In [27]:
t_classifier('You look smart')

[[{'label': 'Ok', 'score': 0.6113644242286682},
  {'label': 'Toxic', 'score': 0.3886356055736542}]]

In [ ]:
# from transformers import pipeline

# clf = pipeline(
#   'text-classification',
#   "ZhengjunHUO/distilbert-toxicity-classifier", 
#   use_fast=True, 
#   #return_all_scores=True
# )